# Local CLaMP

generate CLaMP embeddings for segment comparison

## imports and setup

In [1]:
import os
import torch
import pandas as pd
# from rich import print
from sklearn.metrics.pairwise import cosine_similarity
from midi_player import MIDIPlayer
from midi_player.stylers import dark

In [4]:
s='a_b_t01s03'
x = s.split('_')[-1]
x[4:]

'03'

In [3]:
def get_matches(
    query_tensor: torch.Tensor, df: pd.DataFrame, n: int = 5, mode: str = "best"
) -> list:
    """Finds the names and similarities of the n most similar segments based on cosine similarity.

    Args:
        query_tensor (torch.Tensor): The tensor to compare.
        df (pd.DataFrame): The dataframe containing tensors in the 'embedding' column.
        n (int): number of matches to return
        mode (str): 'worst'

    Returns:
        list: List of tuples containing the name of the row and its cosine similarity.
    """
    query_numpy = query_tensor.reshape(1, -1)
    tensor_matrix = torch.stack(
        [torch.Tensor(tensor) for tensor in df["embedding"]]
    ).numpy()

    similarities = cosine_similarity(query_numpy, tensor_matrix).flatten()
    if mode == "best":
        matches = similarities.argsort()[-n:][::-1]
    elif mode == "worst":
        matches = similarities.argsort()[:n]

    results = [(df.index[i], float(similarities[i])) for i in matches]

    return results

In [4]:
df = pd.read_parquet("test-clamp.parquet")
df.head()

,abc,id,embedding
baba-060-02_0005-0011,"M: 4/4\nL: 1/8\nQ:1/4=120\nK:A\nV:1\nz2 B,3-B,...","[46, 27, 1, 21, 16, 21, 97, 0, 0, 0, 0, 0, 0, ...","[0.5433163, -0.46188298, -0.1765729, 0.0864300..."
bbbb-060-01_0005-0011,"M: 4/4\nL: 1/8\nQ:1/4=120\nK:B\nV:1\nz2 B,3-B,...","[46, 27, 1, 21, 16, 21, 97, 0, 0, 0, 0, 0, 0, ...","[0.88814145, -0.43045604, -0.4846054, 0.208652..."
c4v100-060-03_0000-0005,M: 4/4\nL: 1/8\nQ:1/4=120\nK:C\nV:1\nz2 c6-|c6...,"[46, 27, 1, 21, 16, 21, 97, 0, 0, 0, 0, 0, 0, ...","[0.5935626, -0.27001852, -0.4847664, 0.1845568..."
c4v97-060-03_0005-0011,M: 4/4\nL: 1/8\nQ:1/4=120\nK:C\nV:1\nz2 c6-|c6...,"[46, 27, 1, 21, 16, 21, 97, 0, 0, 0, 0, 0, 0, ...","[0.5935626, -0.27001852, -0.4847664, 0.1845568..."
cascas-060-02_0000-0005,M: 4/4\nL: 1/8\nQ:1/4=120\nK:Bb\nV:1\nz2 C3-C/...,"[46, 27, 1, 21, 16, 21, 97, 0, 0, 0, 0, 0, 0, ...","[0.7008802, -0.57024217, -0.47158375, -0.19007..."


In [5]:
for index in df.index:
    print(index)

baba-060-02_0005-0011
bbbb-060-01_0005-0011
c4v100-060-03_0000-0005
c4v97-060-03_0005-0011
cascas-060-02_0000-0005
cascd-060-02_0000-0005
cascd-060-02_0005-0011
cccc-060-01_0000-0005
leadtest-060-03_0000-0005
trailtest-060-03_0005-0011


In [11]:
es = df["embedding"]
# Print index and content of third element in "es"
print("Index:", es.index[2])
print("Content:", es.iloc[2])


Index: 20240312-080-02_0029-0035_t04s05

Content: [ 7.10535884e-01 -5.31243563e-01  1.84545681e-01  3.23345393e-01
  7.52358496e-01 -3.65603954e-01 -5.22574894e-02 -7.02584147e-01
 -1.36938423e-01 -4.81090486e-01 -9.97825935e-02  6.27783477e-01
 -3.10184240e-01  3.72744679e-01 -5.43351769e-02  5.03280461e-01
 -6.71081424e-01  2.19292760e-01 -4.74930257e-01 -5.50001979e-01
 -1.58497453e-01  2.60973483e-01 -4.32388857e-02  3.52069497e-01
 -7.14530110e-01  3.43954176e-01  5.94730079e-02 -9.00534242e-02
 -1.71551138e-01 -2.45388210e-01  6.02501512e-01  2.91144848e-01
  4.84860361e-01  6.36415720e-01  1.50570944e-01  2.74773717e-01
  4.50650565e-02  4.32523072e-01 -2.45018527e-02 -3.73618379e-02
 -1.92675129e-01 -2.41913453e-01  1.14980713e-02 -2.64815301e-01
  1.20912492e-01  1.64468229e-01 -9.51118708e-01 -7.43044674e-01
 -8.15140679e-02 -1.28373981e-01  5.63710034e-02  2.55559057e-01
  2.48843193e-01  5.47355354e-01  1.61240503e-01 -8.11851397e-02
  3.12514976e-02  2.80842409e-02 -4.81724083e-01  5.02508879e-02
 -7.44745731e-01  1.91393360e-01 -1.14216350e-01  7.77494252e-01
  5.88945627e-01  3.56839746e-02  1.02206564e+00 -5.44879198e-01
  1.27280056e-01 -3.18083942e-01  2.87302494e-01 -2.45028257e-01
 -4.02091220e-02  7.45133102e-01 -3.97513688e-01 -7.80296743e-01
 -3.00457895e-01 -2.63534069e-01 -1.09662652e+00  1.83925815e-02
  4.89092916e-01 -1.44343114e+00 -7.68860161e-01  8.25295150e-01
  7.65722170e-02 -3.73329639e-01 -4.18277204e-01  2.14786872e-01
 -1.14816773e+00  1.45427063e-01  2.97531694e-01  9.07880962e-01
  5.71309388e-01  8.00746799e-01  3.15694094e-01 -2.19256297e-01
 -5.09708941e-01 -7.35472977e-01 -1.37697369e-01  4.03556228e-01
  4.44409937e-01 -3.91735405e-01  3.22800934e-01  8.24813545e-01
 -9.02389407e-01 -6.58263445e-01 -2.41501391e-01  2.15615496e-01
  5.91830015e-01  1.92235678e-01  3.44346344e-01  6.53356850e-01
  6.67562410e-02 -2.04561353e-01  4.53808457e-01 -1.49302438e-01
  9.56272483e-02 -4.26456153e-01  7.01383173e-01  7.60602176e-01
 -2.62148857e-01 -4.82893616e-01 -3.72906029e-01  2.78603137e-01
  1.55815959e-01  1.53027490e-01 -2.91630775e-01 -1.18020087e-01
  6.30783856e-01  1.61767900e-01  4.46893662e-01 -2.38312736e-01
  8.33956599e-01 -1.96860164e-01  1.36281535e-01  2.00477451e-01
  5.77625513e-01  8.77815068e-01 -1.35750934e-01 -2.17477649e-01
  6.27655387e-01  8.97991508e-02 -3.82338434e-01  6.14762902e-02
  2.07649335e-01 -1.33674189e-01  2.68877000e-01  9.93221104e-02
  4.28006053e-02 -7.61183560e-01 -8.45803097e-02 -3.50953013e-01
  7.60891885e-02 -1.30348399e-01 -1.36663660e-01  6.33419216e-01
  1.69729650e-01 -6.79978728e-03 -7.83108920e-02 -3.38259876e-01
  2.84241050e-01 -9.53604400e-01 -4.57891405e-01  9.00592804e-02
  5.05191565e-01  4.06606406e-01 -2.85836995e-01  9.42315534e-02
 -2.41861101e-02  3.35200101e-01  6.46751285e-01 -5.35549879e-01
 -6.29632533e-01  3.21750820e-01  1.48410946e-02 -6.31420985e-02
  2.17393413e-01  6.55348450e-02 -4.67519909e-01 -3.63393694e-01
  4.82722133e-01 -1.94702789e-01 -1.25687492e+00  5.28699994e-01
  5.73592603e-01  3.20196897e-01 -4.23730016e-01 -3.77253443e-02
 -9.74690393e-02  3.18236530e-01 -1.75162047e-01 -4.09572959e-01
 -3.27292234e-01  2.33279169e-02  2.42257081e-02  7.78646588e-01
  1.15725890e-01 -1.17854804e-01 -3.52967203e-01 -1.15952283e-01
 -1.85751274e-01  3.02624881e-01 -5.67957163e-01 -7.31656730e-01
 -3.91134858e-01  5.53131223e-01 -9.03593376e-02  1.70841858e-01
 -4.03555483e-01 -3.63573544e-02  1.27396023e+00 -3.51471931e-01
 -7.52209127e-02 -5.75819373e-01 -5.95849454e-01 -4.04306084e-01
 -8.33541036e-01  6.11595809e-01  3.87537062e-01  1.85597330e-01
 -4.53357756e-01 -1.97806388e-01 -3.25975060e-01  1.97925791e-02
 -6.81344032e-01  2.21809581e-01  3.70674491e-01  2.67036617e-01
  2.74743978e-02 -2.69682854e-01 -2.13904038e-01  1.00128663e+00
 -3.11990470e-01 -5.81522882e-01  1.26061893e+00  5.21516860e-01
 -3.23829651e-01  2.66502082e-01 -1.00946558e+00 -4.22752984e-02
 -5.22032022e-01  4.87415642e-01 -1.07744053e-01  9.09367278e-02
  9.03205335e-01  6.273829

In [31]:
get_matches(df.loc["cccc-060-01_0000-0005_t04s00", "embedding"], df, 20, mode="best")

[('bbbb-060-01_0005-0011_t05s00', 0.9999999403953552),
 ('cccc-060-01_0000-0005_t04s00', 0.9999999403953552),
 ('cccc-060-01_0000-0005_t04s01', 0.9804266691207886),
 ('bbbb-060-01_0005-0011_t05s01', 0.9804266691207886),
 ('cccc-060-01_0000-0005_t03s00', 0.9461910724639893),
 ('bbbb-060-01_0005-0011_t04s00', 0.9461910724639893),
 ('leadtest-060-03_0000-0005_t04s03', 0.9433386325836182),
 ('cccc-060-01_0000-0005_t04s04', 0.9393335580825806),
 ('bbbb-060-01_0005-0011_t05s04', 0.9393335580825806),
 ('bbbb-060-01_0005-0011_t05s02', 0.931435227394104),
 ('cccc-060-01_0000-0005_t04s02', 0.931435227394104),
 ('cccc-060-01_0000-0005_t03s01', 0.9300938248634338),
 ('bbbb-060-01_0005-0011_t04s01', 0.9300938248634338),
 ('baba-060-02_0005-0011_t07s00', 0.929381251335144),
 ('cascas-060-02_0000-0005_t06s00', 0.929381251335144),
 ('cccc-060-01_0000-0005_t06s00', 0.9127424955368042),
 ('bbbb-060-01_0005-0011_t07s00', 0.9127424955368042),
 ('bbbb-060-01_0005-0011_t05s05', 0.9062713384628296),
 ('cccc-

In [35]:
print(df.loc["cccc-060-01_0000-0005_t04s00", "abc"])
MIDIPlayer(
    "../../disklavier/data/datasets/test/train/cccc-060-01_0000-0005_t04s00.mid",
    150,
    styler=dark,
)

M: 4/4
L: 1/8
Q:1/4=60
K:E
V:1
zE2E2E2E-|E

In [36]:
print(df.loc["20240123-070-06_0020-0027_t07s02", "abc"])
MIDIPlayer(
    "../../disklavier/data/datasets/test/train/20240123-070-06_0020-0027_t07s02.mid",
    400,
    styler=dark,
)

M: 4/4
L: 1/8
Q:1/4=70
K:Bb
V:1
z2 z/2D/2G/2A/2 B/2A/2G/2D/2- [DB,]/2A,/2G,/2D,/2| \
G,,/2D,/2G,/2A,/2 B,/2A,/2G,/2D,/2 G,,/2<D,/2G,/2-[A,G,]/2 B,/2A,/2G,/2D,/2| \
G,,/2D,/2G,/2A,/2 B,/2